<a href="https://colab.research.google.com/github/nrfsilva/lista_de_exercicios_01/blob/main/usina_da_pedra_extracao_metricas_indicesDeVegetacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Conectar ao Google Drive***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Importar bibliotecas***

In [2]:
import os
import pandas as pd
import ee

***Google Earth Engine API autenticação***

In [3]:
# autenticação na API
ee.Authenticate()

# iniciar a biblioteca
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4sLY7YLIfd_gXAQOeB7l1mZUMpPCXzjJuVEOM0VPM9I&tc=N5llKWopmqVejeb0-LWRfL5m3FZk_51dlMP2vEjx5Ic&cc=KA7j5gaOx52cElPWwjzbf3TbkaLr5HNwpC5bbdVVE0s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qgQZBUmyEGXPOldEoozjmzzhtj5Ru1UNqr6dPJ9uGsKoFreucr3sXk

Successfully saved authorization token.


***Configurações***

In [15]:
roi = ee.FeatureCollection('users/nildsonsilva/Usina_Pedra/extensao_modelagem_2022_BURITI_UNION_resumido_comReferencias')
talhoes = ee.FeatureCollection('users/nildsonsilva/Usina_Pedra/buriti_2022_sentinel2_10m_pixels_puros_diss')

start_date = '2022-06-01'
end_date = '2022-07-01'

#### LEGENDA DOS ÍNDICES
# ['B4','B8','QA60'] ['NDVI','EVI2','SAVI','OSAVI','ReCl','MSAVI2']
# ['B3','B4','B8','QA60'] ['GNDVI','NDWImcFeeters','GCI','PVR']
# ['B2','B3','B4','B8','QA60'] ['EVI','RBNDVI','SIPI','ARVI','VARI']
# ['B3','B8','B11','B12','QA60'] ['NDSI','NDWIgao','NBR']
# ['B5','B6','B7','B8','QA60'] ['NDRE','NRERI']
########################

interested_area_name = 'buriti' # recomendado inserir o nome minúsculo
reference_year = '2022'
interested_variable = ['GNDVI','NDWImcFeeters','GCI','PVR']
id_column_tables = 'ref_2022' 

filepath_cod_talhao = '/content/drive/MyDrive/RESULTADOS_PEDRA_SENTINEL2_2022/codigo_2022_buriti.csv'
root_path = '/content/drive/MyDrive/USINA_PRODUTIVIDADE/'

input_bands = ['B3','B4','B8','QA60']  # insira a banda de interesse, porém sempre mantenha aqui a B8 (NIR) e a QA60 (BQA), pois elas são utilizadas na remoção das nuvens

dict_bandsname = {
    'B2': 'BLUE',
    'B3': 'GREEN',
    'B4': 'RED',
    'B5': 'REDeDGE1',
    'B6': 'REDeDGE2',
    'B7': 'REDeDGE3',
    'B8': 'NIR',
    'B8A': 'REDeDGE4',
    'B11': 'SWIR1',
    'B12': 'SWIR2',
    'QA60': 'BQA'
}

dict_columns_final_name = {
    'NDVI': 'ndvi',
    'EVI2': 'evi2',
    'SAVI': 'savi',
    'OSAVI': 'osavi',
    'ReCl': 'recl',
    'MSAVI2': 'msavi2',
    'GNDVI': 'gndvi',
    'NDWImcFeeters': 'ndwi_mcfeeters',
    'GCI': 'gci',
    'PVR': 'pvr',
    'EVI': 'evi',
    'RBNDVI': 'rbndvi',
    'SIPI': 'sipi',
    'ARVI': 'arvi',
    'VARI': 'vari',
    'NDSI': 'ndsi',
    'NDWIgao': 'ndwi_gao',
    'NBR': 'nbr',
    'NDRE': 'ndre',
    'NRERI': 'nreri'
}

renamed_bands = [dict_bandsname.get(i) for i in input_bands]
renamed_bands

['GREEN', 'RED', 'NIR', 'BQA']

***Funções necessárias***

In [6]:
# função para padronizar as propriedades
def padronizeProperties(img):

  landsatSpacecrafts = ['LANDSAT_5', 'LANDSAT_7', 'LANDSAT_8']
  copernicusSpacecrafts = ['Sentinel-2A', 'Sentinel-2B']

  img = ee.Image(img)

  landsatImage = img \
    .set('SPACECRAFT', img.get('SPACECRAFT_ID')) \
    .set('CLOUD_COVER', img.get('CLOUD_COVER')) \
    .set('SOLAR_ZENITH_ANGLE', img.get('SUN_ELEVATION')) \
    .set('SOLAR_AZIMUTH_ANGLE', img.get('SUN_AZIMUTH'))

  copernicusImage = img \
    .set('SPACECRAFT', img.get('SPACECRAFT_NAME')) \
    .set('CLOUD_COVER', img.get('CLOUDY_PIXEL_PERCENTAGE')) \
    .set('SOLAR_ZENITH_ANGLE', img.get('MEAN_SOLAR_ZENITH_ANGLE')) \
    .set('SOLAR_AZIMUTH_ANGLE', img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
  
  return ee.Image(ee.Algorithms.If(
      ee.List(landsatSpacecrafts).containsAll([img.get('SPACECRAFT_ID')]),
      landsatImage,
      ee.Algorithms.If(
        ee.List(copernicusSpacecrafts).containsAll([img.get('SPACECRAFT_NAME')]),
        copernicusImage,
        img
      )
  ))

In [7]:
# função de máscara de nuvens para o Sentinel-2
def sentinelCloudMask(img):
  sentinelCloudCoverProbability = 40
  #Identify dark NIR pixels (potential cloud shadow pixels).
  NIR_DRK_THRESH = 0.15
  CLD_PRJ_DIST = 2
  BUFFER = 50

  s2CloudsImage = ee.Image(ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
                           .filterMetadata('system:index', 'equals', img.get('system:index')).first()) \
                           .unmask()

  #1: clouds 0: non-clouds.
  isClouds = s2CloudsImage.gte(sentinelCloudCoverProbability)
  darkPixels = ee.Image(img.select('NIR').lt(NIR_DRK_THRESH)).rename('dark_pixels')

  #Determine the direction to project cloud shadow from clouds (assumes UTM projection).
  shadowAzimuth = ee.Number(90).subtract(ee.Number(img.get('SOLAR_AZIMUTH_ANGLE')))
  
  #Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
  isCloudsProj = isClouds \
    .directionalDistanceTransform(shadowAzimuth, CLD_PRJ_DIST*10) \
    .reproject(**{'crs': img.select(0).projection(), 'scale': 100}) \
    .select('distance').mask() \
    .rename('cloud_transform')

  #Identify the intersection of dark pixels with cloud shadow projection.
  isShadows = isCloudsProj.multiply(darkPixels).rename('shadows')

  #Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
  cloudsOrCloudShadows = isClouds.Or(isShadows)

  #Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input. 
  #20 m scale is for speed, and assumes clouds don't require 10 m precision.
  cloudsOrCloudShadows = cloudsOrCloudShadows \
    .focal_min(2) \
    .focal_max(BUFFER*2/20) \
    .reproject(**{'crs': img.select([0]).projection(), 'scale': 20}) \
    .rename('cloudmask')

  return ee.Image(cloudsOrCloudShadows)
    

In [8]:
# remover pixels fora da escala padrão
def mask_invalid_values(img):
  img = ee.Image(img)
  mask = img.gt(10000.0000000001)

  band = img \
    .where(img.gte(0.0000000001),img) \
    .where(img.lt(0.0000000001),0) \
    .where(img.gt(10000.0000000001),img.updateMask(mask.eq(0)))

  return band

In [9]:
# função de pré-processamento
def basic_process(img):

  img = ee.Image(img)

  img_edit = img.clip(roi.geometry())

  year = ee.String(img.get('system:index')).slice(0,4)
  month = ee.String(img.get('system:index')).slice(4,6)
  day = ee.String(img.get('system:index')).slice(6,8)

  date_string = ee.String(year.cat(month).cat(day))

  img_edit = padronizeProperties(img_edit)
  cloudMask = ee.Image(sentinelCloudMask(img_edit))

  img_masked = ee.Image(img_edit.updateMask(cloudMask.eq(0)))

  interested_bands = renamed_bands[0:-1]

  stack = ee.ImageCollection([mask_invalid_values(img_masked.select(ee.String(i))) for i in interested_bands]).toBands()
  stack = stack.select(stack.bandNames(),interested_bands)

  return stack \
    .int16() \
    .copyProperties(img) \
    .set({'data_string':date_string})

In [10]:
# função para cálculo dos índices de vegetação
def obtain_vis_values(data):
  col = ee.ImageCollection(s2Collection_v1).filterMetadata('data_string','equals',data)
  imgReferencia = ee.Image(col.first())
  
  data_string = ee.String(imgReferencia.get('data_string'))
  col_mean = ee.Image(col.mean())

  img = ee.Image(col_mean).divide(ee.Image(10000))
  variables = renamed_bands[0:-1]

  if (('RED' in variables) and ('NIR' in variables) and len(variables) == 2) == True:

    img = ee.Image(img)

    ndvi = ee.Image(0).expression('((NIR - RED) / (NIR + RED))',{
         'NIR': img.select('NIR'),
         'RED': img.select('RED')})
    ndvi = ee.Image(ndvi.updateMask(ndvi.gte(0.0000000001))).rename('NDVI')

    evi2 = ee.Image(0).expression('2.5*((NIR - RED)/(NIR + 2.4*RED + 1))',{
        'NIR': img.select('NIR'),
        'RED': img.select('RED')
     })
    evi2 = evi2.updateMask(evi2.gte(0.0000000001)).rename('EVI2')

    savi = ee.Image(0).expression('((NIR-RED)*(1+L))/((NIR+RED)+L)',{
         'NIR': img.select('NIR'),
         'RED': img.select('RED'),
         'L': ee.Image(0.5)
      }).rename('SAVI')

    osavi = ee.Image(0).expression('(NIR-RED)/(NIR+RED+0.16)',{
         'NIR': img.select('NIR'),
         'RED': img.select('RED'),
      }).rename('OSAVI')

    recl = ee.Image(0).expression('(NIR/RED)-1',{
         'NIR': img.select('NIR'),
         'RED': img.select('RED'),
      }).rename('ReCl')

    msavi2 = ee.Image(0).expression('((2 * NIR + 1)-sqrt(((2*NIR+1)**2)-(8*(NIR-RED))))/2',{
         'NIR': img.select('NIR'),
         'RED': img.select('RED'),
      }).rename('MSAVI2')

    stack = ndvi.addBands(evi2).addBands(savi).addBands(osavi).addBands(recl).addBands(msavi2)

    return stack.copyProperties(imgReferencia).set('data_string',data_string)
    
  elif (('GREEN' in variables) and ('RED' in variables) and ('NIR' in variables) and len(variables) == 3) == True:

    img = ee.Image(img)

    gndvi = ee.Image(0).expression('(NIR - GREEN) / (NIR + GREEN)',{
         'GREEN': img.select('GREEN'),
         'NIR': img.select('NIR'),
     }).rename('GNDVI')

    ndwi_mcfeeters = ee.Image(0).expression('(GREEN-NIR)/(GREEN+NIR)',{
        'GREEN': img.select('GREEN'),
         'NIR': img.select('NIR'),
     }).rename('NDWImcFeeters')
     
    gci = ee.Image(0).expression('(NIR/GREEN)-1',{
         'GREEN': img.select('GREEN'),
         'NIR': img.select('NIR'),
     }).rename('GCI')

    pvr = ee.Image(0).expression('(GREEN - RED) / (GREEN + RED)',{
         'GREEN': img.select('GREEN'),
         'RED': img.select('RED'),
     }).rename('PVR') 

    stack = gndvi.addBands(ndwi_mcfeeters).addBands(gci).addBands(pvr)

    return stack.copyProperties(imgReferencia).set({'data_string':data_string})

  elif (('BLUE' in variables) and ('GREEN' in variables) and ('RED' in variables) and ('NIR' in variables) and len(variables) == 4) == True:

    img = ee.Image(img)

    evi = ee.Image(0).expression('G*((NIR - RED)/(NIR +C1*RED -C2*BLUE + L))',{
         'BLUE': img.select('BLUE'),
         'RED': img.select('RED'),
         'NIR': img.select('NIR'),
         'G': ee.Image(2.5),
         'C1': ee.Image(6),
         'C2': ee.Image(7.5),
         'L': ee.Image(1)
     })
    evi = evi.updateMask(evi.gte(0.0000000001))
    evi = evi.updateMask(evi.lte(1.0000000001)).rename('EVI')

    rbndvi = ee.Image(0).expression('(NIR - (RED + BLUE)) / (NIR + (RED + BLUE))',{
         'BLUE': img.select('BLUE'),
         'RED': img.select('RED'),
         'NIR': img.select('NIR'),
     }).rename('RBNDVI')

    sipi = ee.Image(0).expression('(NIR-BLUE) / (NIR-RED)',{
         'BLUE': img.select('BLUE'),
         'RED': img.select('RED'),
         'NIR': img.select('NIR'),
     }).rename('SIPI')

    arvi = ee.Image(0).expression('(NIR-2*RED+BLUE) / (NIR+2*RED+BLUE)',{
         'BLUE': img.select('BLUE'),
         'RED': img.select('RED'),
         'NIR': img.select('NIR'),
    }).rename('ARVI')

    vari = ee.Image(0).expression('(GREEN - RED) / (GREEN + RED - BLUE)',{
         'BLUE': img.select('BLUE'),
         'GREEN': img.select('GREEN'),
         'RED': img.select('RED'),
     }).rename('VARI')

    stack = evi.addBands(rbndvi).addBands(sipi).addBands(arvi).addBands(vari)

    return stack.copyProperties(imgReferencia).set({'data_string':data_string})

  elif (('GREEN' in variables) and ('NIR' in variables) and ('SWIR1' in variables) and ('SWIR2' in variables) and len(variables) == 4) == True:

    img = ee.Image(img)

    ndsi = ee.Image(0).expression('(GREEN - SWIR1)/(GREEN + SWIR1)',{
         'GREEN': img.select('GREEN'),
         'SWIR1': img.select('SWIR1'),
     }).rename('NDSI')

    ndwi_gao = ee.Image(0).expression('(NIR-SWIR1)/(NIR+SWIR1)',{
         'NIR': img.select('NIR'),
         'SWIR1': img.select('SWIR1'),
     }).rename('NDWIgao')

    nbr = ee.Image(0).expression('(NIR-SWIR2)/(NIR+SWIR2)',{
         'NIR': img.select('NIR'),
         'SWIR2': img.select('SWIR2'),
     }).rename('NBR')

    stack = ndsi.addBands(ndwi_gao).addBands(nbr)

    return stack.copyProperties(imgReferencia).set({'data_string':data_string})

  elif (('REDeDGE1' in variables) and ('REDeDGE2' in variables) and ('REDeDGE3' in variables) and ('NIR' in variables) and len(variables) == 4) == True:

    img = ee.Image(img)

    ndre = ee.Image(0).expression('(REDeDGE3-REDeDGE1)/(REDeDGE3+REDeDGE1)',{
         'REDeDGE1': img.select('REDeDGE1'),
         'REDeDGE3': img.select('REDeDGE3'),
     }).rename('NDRE')

    nreri = ee.Image(0).expression('(NIR-REDeDGE2)/(NIR+REDeDGE1)',{
         'REDeDGE1': img.select('REDeDGE1'),
         'REDeDGE2': img.select('REDeDGE2'),
         'NIR': img.select('NIR'),
    }).rename('NRERI')

    stack = ndre.addBands(nreri)

    return stack.copyProperties(imgReferencia).set({'data_string':data_string})

  else:
    print('Revise as bandas espectrais de entrada.')

***Processamento no GEE***

In [16]:
# filtrar imagens Sentinel-2
s2Collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterBounds(roi).filterDate(start_date,end_date).select(input_bands,renamed_bands)

In [17]:
# pré-processamento e obterção de uma lista com as datas das imagens
s2Collection_v1 = ee.ImageCollection(s2Collection.map(basic_process))

listaDatas = s2Collection_v1.aggregate_array('data_string').distinct()

In [18]:
# calcular índices de vegetação
s2Collection_v2 = ee.ImageCollection(listaDatas.map(obtain_vis_values))

*Exportar asset para a conta do GEE, rodar apenas se não é possível exportar o CSV para janelas de tempo muito longas. Após exportar para o drive e concluir essa etapa, por favor, pular para o tópico referente ao ajuste da tabela para o padrão consumido pelo modelo de produtividade.*

In [ ]:
# função de ajuste do nome das bandas
def change_name_v2(list_elem):
  nameList = ee.String(list_elem).split('_')
  return ee.String(nameList.get(1)).cat('_').cat(nameList.get(2))

In [ ]:
for i in interested_variable:
  i_str = ee.String(i)
  s2Collection_v2_filtered = s2Collection_v2.select(i_str).map(lambda img: img.rename(ee.String(i_str).cat('_').cat(ee.String(img.get('data_string')))))
  s2Collection_v3 = s2Collection_v2_filtered.toBands()

  # nomes finais das bandas do acumulado da radiação global
  s2Collection_v3_bandsName = s2Collection_v3.bandNames()
  s2Collection_v3_bandsName_new = s2Collection_v3_bandsName.map(change_name_v2)

  s2Collection_v4 = s2Collection_v3.select(s2Collection_v3_bandsName,s2Collection_v3_bandsName_new)

  taskname = 'gee_'+interested_area_name+'_'+reference_year+'_'+i

  task_band = ee.batch.Export.image.toAsset(**{
    'image': s2Collection_v4,
    'description': taskname,
    'assetId': 'users/nildsonsilva/Usina_Pedra/temp_imagens_indices_vegetacao/'+taskname,
    'scale': 10,
    'region': roi.geometry()
  })
  task_band.start()

In [ ]:
for i in interested_variable:
  taskname = 'gee_'+interested_area_name+'_'+reference_year+'_'+i

  result = ee.Image(ee.String('users/nildsonsilva/Usina_Pedra/temp_imagens_indices_vegetacao/'+taskname))

  # obter para a banda espectral de interesse sua média para o talhão
  poligonos_metricas = result.reduceRegions(**{
      'collection': talhoes,
      'reducer': ee.Reducer.mean(),
      'crs': 'EPSG:3857',
      'scale': 10,
      'tileScale': 16,
    })

  # exportar a FeatureCollection do GEE para um CSV
  output_path = 'USINA_PRODUTIVIDADE'

  task = ee.batch.Export.table.toDrive(**{
    'collection': ee.FeatureCollection(poligonos_metricas),
    'description': taskname,
    'fileFormat': 'CSV',
    'folder': output_path,
  })
  task.start()

*Caso sua série temporal não seja muito longa, rodar o script a partir daqui*

In [19]:
# exportar as tabelas com os resultados do GEE por variável de interesse
for i in interested_variable:
  i_str = ee.String(i)
  s2Collection_v2_filtered = s2Collection_v2.select(i_str).map(lambda img: img.rename(ee.String(i_str).cat('_').cat(ee.String(img.get('data_string')))))
  result = s2Collection_v2_filtered.toBands()

  poligonos_metricas = result.reduceRegions(**{
    'collection': talhoes,
    'reducer': ee.Reducer.mean(),
    'crs': 'EPSG:3857',
    'scale': 10,
    'tileScale': 16,
  })
  
  taskname = 'gee_'+interested_area_name+'_'+reference_year+'_'+i
  output_path = 'USINA_PRODUTIVIDADE'

  # Export the FeatureCollection to a CSV file.
  task = ee.batch.Export.table.toDrive(**{
    'collection': ee.FeatureCollection(poligonos_metricas),
    'description': taskname,
    'fileFormat': 'CSV',
    'folder': output_path,
  })
  task.start()


***Ajustar as tabelas exportadas pelo GEE para o padrão requerido pelo modelo e produtividade para cana-de-açúcar***  

In [ ]:
# abrir e ajustar o CSV com o código do talhão atribuido pela Agrosatélite e seus respectivos códigos anuais definidos pela Usina
cod_talhao = pd.read_csv(filepath_cod_talhao,delimiter=',')
cod_talhao = cod_talhao[['ref_2022','y2015','y2016','y2017','y2018','y2019','y2020','y2021','y2022']]
cod_talhao

,ref_2022,y2015,y2016,y2017,y2018,y2019,y2020,y2021,y2022
0,1,,30687009,30687009,30687009,,30687009,30687009,NaN
1,2,,30687007,30687007,30687007,,30687007,30687007,NaN
2,3,,,30948001,30948004,30948004,30948004,40903004,NaN
3,4,,30948001,30948001,,30948004,30948004,40903004,NaN
4,5,,30832001,30832001,,30832001,30832001,30832001,NaN
...,...,...,...,...,...,...,...,...,...
76877,76878,,30687006,30687006,30687006,,30687006,30687006,NaN
76878,76879,,30687003,30687003,30687003,,30687003,30687003,NaN
76879,76880,,30687004,30687004,30687004,,30687004,30687004,NaN
76880,76881,,30687001,30687001,30687001,,30687001,30687001,NaN


In [ ]:
final_vars = ['NDVI','EVI2','SAVI','OSAVI','ReCl','MSAVI2','GNDVI','NDWImcFeeters','GCI','PVR','EVI','RBNDVI','SIPI','ARVI','VARI','NDSI','NDWIgao','NBR','NDRE','NRERI']

In [ ]:
# ajustar as tabelas e exportar os resultados finais
for i in final_vars:
  var_endswith = i+'.csv'
  print("Variável em processamento: "+i)

  filename_from_gee = 'gee_'+interested_area_name+'_'+reference_year+'_'+var_endswith
  filepath_table_from_gee = root_path+filename_from_gee

  df = pd.read_csv(filepath_table_from_gee)

  str_contains = id_column_tables+'|'+i

  df_edit1 = df[df.columns[df.columns.str.contains(str_contains)]]

  df_edit3 = pd.melt(df_edit1, id_vars=[id_column_tables], 
            value_name=i).sort_values(i) # transformar todas as colunas referentes a banda de interesse em uma única coluna

  df_edit3['data'] = df_edit3['variable'].str.rsplit("_", n=1, expand=True)[1]

  column_final_name = dict_columns_final_name.get(i)

  df_edit3 = df_edit3.rename(columns = { 
        i: column_final_name,
  }, inplace = False)

  ordem = [id_column_tables,'data',column_final_name]

  df_new = df_edit3[ordem]

  df_final = df_new.merge(cod_talhao, on=id_column_tables, how="left")

  df_final = df_final.rename(columns = { 
      id_column_tables: 'id',
  }, inplace = False)

  filename = interested_area_name+'_'+reference_year+'_'+i+'.csv'

  # exportar csv
  new_path = root_path+'OUTPUT'
  final_name = new_path + '/' + filename
  df_final.to_csv(final_name,columns=df_final.columns,index=False)
  print("O arquivo {} foi exportado com sucesso!".format(filename))